In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
from skimage.io import imread
import skimage.measure as skimeas
from os.path import join
from mayavi import mlab
from skimage import measure
%matplotlib inline  

In [2]:
filename = r'cells.tif'   
dirpath = r'' 
filepath = join(dirpath, filename) #final path to load image
img = imread(filepath) #loading image           

In [3]:
def myplt(img, title):
    """This function displays the input image with interpolation='none', cmap='magma' in the range of 0-255 (8-bits). """
    plt.imshow(img,interpolation='none',cmap='magma', vmin=0, vmax=255)
    plt.colorbar(fraction=0.046, pad=0.04) 
    plt.title(title)

In [4]:
# Function to plot pairs of images
def pltPair(img1, img2, title1, title2, cmap1, cmap2, lim):
    """This function displays a pair of input image with interpolation='none',in the range of lims. """

    plt.figure(figsize=(16,16))
    plt.subplot(1,2,1)
    plt.imshow(img1, interpolation='none', cmap=cmap1, vmin=lim[0], vmax=lim[1])
    plt.title(title1)
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.subplot(1,2,2)
    plt.imshow(img2, interpolation='none', cmap=cmap2, vmin=lim[2], vmax=lim[3])
    plt.title(title2)
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.show()

In [5]:
# Plot image and diplay image features
imgDim = img.shape
nPlanes = imgDim[0]
from ipywidgets import interact
@interact (z=(0,nPlanes-1,1))
def plot_slides (z):
  myplt(img[z,:,:], "Membranes")

print('Variable Type: ', type(img))
print('Image data type: ', img.dtype)
print('Image dimension: ', img.shape)
plt.figure(figsize=(20,20))
plt.show()

interactive(children=(IntSlider(value=153, description='z', max=306), Output()), _dom_classes=('widget-interac…

Variable Type:  <class 'numpy.ndarray'>
Image data type:  uint16
Image dimension:  (307, 786, 712)


<Figure size 1440x1440 with 0 Axes>

In [6]:
# Create an image border mask

border_mask = np.zeros(img.shape, dtype=bool)
border_mask = ndi.binary_dilation(border_mask, border_value=1)

# Remove the cells at the border

# Create a copy of the initial result
clean_img = np.copy(img)

for cell_ID in np.unique(img):

    # Create a mask that contains only the 'current' cell of the iteration
    cell_mask = img ==cell_ID 
    
    # Use the cell mask and the border mask to test if the cell has pixels touching 
    # the image border or not.
    cell_border_overlap = np.logical_and(cell_mask, border_mask)  # Overlap of cell mask and boundary mask
    total_overlap_pixels = np.sum(cell_border_overlap)            # Sum overlapping pixels

    # If a cell touches the image boundary, delete it by setting its pixels in the segmentation to 0.
    if total_overlap_pixels > 0: 
        clean_img[cell_mask] = 0

# Re-label the remaining cells to keep the numbering consistent from 1 to N (with 0 as background).

for new_ID, cell_ID in enumerate(np.unique(clean_img)[1:]):  # The [1:] excludes 0 from the list (background)!
    clean_img[clean_img==cell_ID] = new_ID+1                  # The same here for the +1
        
nCells = clean_img.max();
print(str(nCells), ' cells detected after removing the ones at the border')

249  cells detected after removing the ones at the border


In [7]:
@interact(z=(1,nPlanes-1,1))
def plot_slides(z):
  pltPair(img[z,:,:], clean_img[z,:,:], 'Cells initial', 'Cells final', 'jet', 'jet', [0,255,0,nCells])
  plt.show()

interactive(children=(IntSlider(value=153, description='z', max=306, min=1), Output()), _dom_classes=('widget-…

In [8]:
# Create cell edge mask
edges = np.zeros_like(clean_img)

# Iterate over the cell IDs
for cell_ID in np.unique(clean_img)[1:nCells]:#here we can choose how many cells we want to calculate

    # Erode the cell's mask by 1 pixel using 'ndi.binary_erode'
    cell_mask = clean_img==cell_ID
    eroded_cell_mask = ndi.binary_erosion(cell_mask, iterations=1) # 
    
    # Create the cell edge mask using 'np.logical_xor'
    edge_mask = np.logical_xor(cell_mask, eroded_cell_mask)
    
    # Add the cell edge mask to the empty array generated above, labeling it with the cell's ID
    edges[edge_mask] = cell_ID

In [9]:
# Vizualize   
maskEdges = np.ma.array(edges, mask=edges==0)
@interact(z=(1,nPlanes-1,1))
def plot_slides(z):
  pltPair(img[z,:,:], maskEdges[z,:,:], 'Smooth image', 'Cell edges', 'gray', 'prism', [0,255,0,nCells])
  plt.show()

interactive(children=(IntSlider(value=153, description='z', max=306, min=1), Output()), _dom_classes=('widget-…

In [10]:
# Create a dictionary that contains a key-value pairing for each measurement

results = {"cell_id"      : [],
           "cell_area"    : [],
           "cell_edge"    : []}


# Run calculations for each cell

for cell_id in np.unique(clean_img)[1:nCells]:

    # Mask the current cell and cell edge
    cell_mask = clean_img==cell_id
    edge_mask = edges==cell_id
    
    # Get the measurements
    results["cell_id"].append(cell_id)
    results["cell_area"].append(np.sum(cell_mask))
    results["cell_edge"].append(np.sum(edge_mask))


In [15]:
# Print the results 
for key in results.keys(): 
    print(key + ":", results[key][:nCells], '\n')

cell_id: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220

In [58]:
verts, faces, normals, values = measure.marching_cubes(maskEdges, 0.0) 

In [60]:
x= [vert[0] for vert in verts]
y= [vert[1] for vert in verts]
z= [vert[2] for vert in verts]

In [62]:
mlab.triangular_mesh(x,y,z, faces, representation = 'wireframe')
mlab.show() 